In [5]:
import cv2
import mediapipe as mp
import pyautogui
import threading

In [7]:
import cv2
import mediapipe as mp
import pyautogui
import threading

# Initialize Mediapipe Hands and Drawing
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Initialize the video capture
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
height, width, _ = frame.shape

# Initialize a variable to store the current frame
current_frame = None

# Define the DPI scaling factor
DPI_SCALE = 2.0  # Increase this value to increase cursor sensitivity

# Function to continuously capture frames from the webcam
def capture_frames():
    global current_frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = frame

# Start the frame capture thread
capture_thread = threading.Thread(target=capture_frames)
capture_thread.start()

while True:
    if current_frame is not None:
        # Flip the frame horizontally for a later selfie-view display
        image = cv2.flip(current_frame, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the frame and get hand landmarks
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract landmarks for the index finger tip and thumb tip
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

                # Convert normalized coordinates to screen coordinates
                screen_width, screen_height = pyautogui.size()
                x = int(index_finger_tip.x * screen_width * DPI_SCALE)
                y = int(index_finger_tip.y * screen_height * DPI_SCALE)

                # Ensure the cursor does not move out of the screen bounds
                x = max(0, min(screen_width - 1, x))
                y = max(0, min(screen_height - 1, y))

                # Move the mouse
                pyautogui.moveTo(x, y)

                # Check distance between index finger tip and thumb tip
                distance = ((index_finger_tip.x - thumb_tip.x) ** 2 + (index_finger_tip.y - thumb_tip.y) ** 2) ** 0.5

                # Click if distance is less than a threshold
                if distance < 0.05:
                    pyautogui.click()

        # Display the frame
        cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(1) & 0xFF == ord("q"):  # Exit on pressing 'Esc'
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

d:\Virtual_mouse\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
